In [1]:
import numpy as np
import faiss

x = np.random.rand(10000, 10).astype("float32")
ncentroids = 1024
niter = 20
verbose = True
d = x.shape[1]
kmeans = faiss.Kmeans(d, ncentroids, niter=niter, verbose=verbose)
kmeans.train(x)

Loading faiss with AVX2 support.


1648.4741

In [5]:
from pathlib import Path
import h5py
from helper import load_data

basename = "indochina-2004"
model_path = Path("/data/models") / basename
print("Loading data..")
X, Y = load_data(model_path)
classes = len(np.unique(Y))
print("X shape: {}".format(X.shape))

Loading data..
X shape: (7414768, 64)


In [11]:
d = X.shape[1]
ncentroids = 5
kmeans = faiss.Kmeans(d, ncentroids, niter=50, verbose=verbose)
kmeans.train(X)

18.094744

In [13]:
D, I = kmeans.index.search(X, 1)

In [20]:
I[:20]

array([[0],
       [2],
       [2],
       [2],
       [2],
       [0],
       [2],
       [4],
       [2],
       [0],
       [0],
       [0],
       [4],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

In [25]:
import json
from helper import iter_partitions

entities_list = []
for json_f, _ in iter_partitions(model_path, names=True):
        with open(json_f, "rt") as f:
            entities = json.load(f)
        entities_list += [int(i) for i in entities]

In [26]:
entities_list[:10]

[5529548,
 2085837,
 3864810,
 4536470,
 5151715,
 4550773,
 5574743,
 1748777,
 1492777,
 3389487]

In [35]:
import linecache

urls_file = Path('/data/graphs/') / basename / (basename + '.urls')
f = urls_file.as_posix()
for pos, value in enumerate(entities_list):
    print(I[pos], linecache.getline(f, value + 1))
    if pos >= 20:
        break

[0] http://www.hcm.edu.vn/cityweb/GV_Mon.idc?vfile=13EA01GV&mon=GD

[2] http://www.mofa.gov.vn:8080/tbqt/15,04/ttqg%20hoso15,04.htm

[2] http://www.buysell.in.th/buysell/cgi-bin/buysell.cgi?ct=jobother&md=second&id=1858&min=&max=

[2] http://ipc10.dip.go.th/CalendarActivity/?tdate=13%2F5%2F2548

[2] http://intraserver.nurse.cmu.ac.th/mis/course/course_detail.asp?CourseNo=559712&Offset=lecture

[0] http://www.jinan.co.th/?sid=935820695&m=2&productid=13PC   Baby K6-333 Mhz.

[2] http://www.pcworld.com.vn/pcworld/niengiam/?tgt=Detail.asp&UnitID=CT0170&_return=L3Bjd29ybGQvbmllbmdpYW0vRGVmYXVsdC5hc3A~dGd0PVNlYXJjaC5hc3AmQ2xhc3NpZmllZD0xJmlsPUgg

[4] http://www.gdt.gov.vn/index.jsp?sid=7&scr=800&progid=25025&Sohieu=1040000006&Dieukien=HDON&ten=H%C3%B3a %C4%91%C6%A1n ch%E1%BB%A9ng t%E1%BB%AB&Type=1

[2] http://www.bagan.net.mm/news/new_detail.asp?news_no=2155

[0] http://www.krirk.ac.th/webboards/AdminControl/AEdit.asp?qNo=98Cat5&QCat=5&AnswerID=14

[0] http://www.thaitravel.co.th/?page=SdbsH

In [38]:
help(mat)

Help on PCAMatrix in module faiss.swigfaiss_avx2 object:

class PCAMatrix(LinearTransform)
 |  Method resolution order:
 |      PCAMatrix
 |      LinearTransform
 |      VectorTransform
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __del__ lambda self
 |  
 |  __getattr__ lambda self, name
 |  
 |  __init__(self, d_in=0, d_out=0, eigen_power=0, random_rotation=False)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__ = _swig_repr(self)
 |  
 |  __setattr__ lambda self, name, value
 |  
 |  __swig_destroy__ = delete_PCAMatrix(...)
 |  
 |  apply_py = apply_method(self, x)
 |  
 |  copy_from(self, other)
 |  
 |  prepare_Ab(self)
 |  
 |  train = replacement_vt_train(self, x)
 |  
 |  train_c = train(self, n, x)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  PCAMat
 |  
 |  balanced_bins
 |  
 |  eigen_power
 |  
 |  eigenvalues
 |  
 |  max_points_per_d
 |

In [39]:
mat = faiss.PCAMatrix (64, 2)
mat.train(X)
assert mat.is_trained
tr = mat.apply_py(X)
tr.shape

(7414768, 2)

In [40]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'